In [1]:
import pandas as pd
import requests
import json
from tqdm import tqdm
import warnings
warnings.filterwarnings(action='ignore') 

data = pd.read_pickle('/Users/iseunghyeon/Desktop/프로젝트/data/버스정류장/Bus_around_500_08231838.pickle')

In [15]:
outer = pd.read_pickle('/Users/iseunghyeon/Desktop/프로젝트/data/버스정류장/Bus_Main0823.pickle')
cols = ['정류장번호'] + list(outer.columns[outer.columns.get_loc('공원거리구분_A'):])
outer_cp = outer[cols].copy()
outer_cp

,정류장번호,공원거리구분_A,공원거리구분_B,CH거리구분_A,CH거리구분_B,하천거리구분_A,하천거리구분_B
0,ASB285000077,0,0,5,25,1,0
1,ASB285000078,0,0,11,15,0,1
2,ASB285000079,0,0,12,22,0,0
3,ASB285000094,0,0,5,24,1,0
4,ASB285000095,0,1,11,16,0,1
...,...,...,...,...,...,...,...
2309,GGB285000034,1,0,11,16,0,1
2310,GGB285000035,0,1,11,9,0,0
2311,GGB285500010,4,4,17,31,0,0
2312,GGB285500020,6,8,14,62,0,0


In [16]:
merge1 = pd.merge(data,outer_cp,on='정류장번호',how='left')
len(data)==len(merge1)


True

In [17]:
merge1

,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,FD6,CE7,HP8,PM9,공원거리구분_A,공원거리구분_B,CH거리구분_A,CH거리구분_B,하천거리구분_A,하천거리구분_B
0,ASB285000077,성환8리,36.921468,127.122774,0,2,1,0,0,2,...,8,0,1,0,0,0,5,25,1,0
1,ASB285000078,송덕,36.923783,127.118976,0,2,2,1,0,2,...,3,0,0,0,0,0,11,15,0,1
2,ASB285000079,국립축산과학원,36.925931,127.111136,0,1,1,1,0,0,...,0,0,0,0,0,0,12,22,0,0
3,ASB285000094,성환8리,36.921479,127.122505,0,2,1,0,0,2,...,7,0,0,0,0,0,5,24,1,0
4,ASB285000095,성환터미널,36.917221,127.132732,1,11,4,3,30,3,...,205,0,16,0,0,1,11,16,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2309,GGB285000034,성환터미널,36.916583,127.132933,1,12,4,3,30,3,...,0,32,0,0,1,0,11,16,0,1
2310,GGB285000035,남서울대학,36.910267,127.142200,0,5,1,1,1,8,...,0,9,0,0,0,1,11,9,0,0
2311,GGB285500010,천안TG.단대.천안대,36.822650,127.162867,0,12,3,1,37,8,...,0,27,0,0,4,4,17,31,0,0
2312,GGB285500020,천안종합터미널,36.819833,127.155733,2,34,16,0,115,22,...,0,166,0,0,6,8,14,62,0,0


In [27]:
target_list = list(merge1.columns[merge1.columns.get_loc('MT1'):merge1.columns.get_loc('PM9')+1])
target_list
merge1['sum']=merge1[target_list].sum(axis=1)
merge1

,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,CE7,HP8,PM9,공원거리구분_A,공원거리구분_B,CH거리구분_A,CH거리구분_B,하천거리구분_A,하천거리구분_B,sum
0,ASB285000077,성환8리,36.921468,127.122774,0,2,1,0,0,2,...,0,1,0,0,0,5,25,1,0,17
1,ASB285000078,송덕,36.923783,127.118976,0,2,2,1,0,2,...,0,0,0,0,0,11,15,0,1,12
2,ASB285000079,국립축산과학원,36.925931,127.111136,0,1,1,1,0,0,...,0,0,0,0,0,12,22,0,0,3
3,ASB285000094,성환8리,36.921479,127.122505,0,2,1,0,0,2,...,0,0,0,0,0,5,24,1,0,15
4,ASB285000095,성환터미널,36.917221,127.132732,1,11,4,3,30,3,...,0,16,0,0,1,11,16,0,1,280
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2309,GGB285000034,성환터미널,36.916583,127.132933,1,12,4,3,30,3,...,32,0,0,1,0,11,16,0,1,92
2310,GGB285000035,남서울대학,36.910267,127.142200,0,5,1,1,1,8,...,9,0,0,0,1,11,9,0,0,26
2311,GGB285500010,천안TG.단대.천안대,36.822650,127.162867,0,12,3,1,37,8,...,27,0,0,4,4,17,31,0,0,116
2312,GGB285500020,천안종합터미널,36.819833,127.155733,2,34,16,0,115,22,...,166,0,0,6,8,14,62,0,0,444


In [29]:
score_df = merge1.copy()
# score 칼럼 구하기
score_df['score'] = score_df['sum'].astype('int') + (score_df['공원거리구분_A']*2 + score_df['공원거리구분_B']) + (score_df['CH거리구분_A'] *2 + score_df['CH거리구분_B']) + (score_df['하천거리구분_A'] *2 + score_df['하천거리구분_B'])
# score_df[['주변시설의총합','score']]
sorted_score_df = score_df.sort_values(by='score',ascending=False)
sorted_score_df

# sorted_score_df =score_df[['정류장번호','정류장명','위도','경도','도로명주소','시','구','읍/면/동','리','score']].sort_values(by='score',ascending=False)
# sorted_score_df


,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,HP8,PM9,공원거리구분_A,공원거리구분_B,CH거리구분_A,CH거리구분_B,하천거리구분_A,하천거리구분_B,sum,score
856,CAB285000936,성정중학교,36.830708,127.137961,1,66,1,1,25,27,...,32,0,5,10,4,47,0,0,1145,1220
174,ASB288001967,불당상업지구입구,36.811821,127.108607,1,27,11,0,270,36,...,74,0,2,7,0,3,1,0,1201,1217
857,CAB285000937,성정중학교,36.829957,127.137722,2,61,1,1,16,25,...,50,0,5,9,3,60,0,0,1098,1183
855,CAB285000935,두정양돈농협,36.833573,127.138038,1,54,9,3,70,19,...,21,0,5,7,7,28,0,0,1085,1144
924,CAB285001024,두정도서관,36.833743,127.135533,1,49,13,5,79,18,...,22,0,7,6,8,23,0,0,1069,1128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2164,CAB285071671,의관동종점,36.822265,127.326065,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2017,CAB285071513,보산원1리,36.690781,127.074633,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
521,CAB285000499,덕전1리,36.809343,127.221424,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
524,CAB285000502,덕전1리(용소),36.808634,127.224933,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
# import pandas as pd
# import folium



# # Load your DataFrame (ctc_df)
# # ctc_df = pd.read_csv('your_data.csv')
# ctc_df = sorted_score_df.head(188)

# # Convert latitude and longitude columns to float if needed
# ctc_df['위도'] = ctc_df['위도'].astype(float)
# ctc_df['경도'] = ctc_df['경도'].astype(float)

# # Calculate the mean latitude and longitude for centering the map
# center_lat = ctc_df['위도'].mean()
# center_lon = ctc_df['경도'].mean()

# # Create a Folium map centered at the mean location
# m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# # Add markers for each latitude and longitude pair in your DataFrame
# for index, row in ctc_df.iterrows():
#     folium.Marker(
#         location=[row['위도'], row['경도']],
#         popup=row['정류장명'],  # Optional popup information
#         icon=folium.Icon(color='blue')  # Optional icon customization
#     ).add_to(m)

# # Save the map to an HTML file
# path = '/Users/iseunghyeon/Desktop/프로젝트/data/folium시각화/{when}.html'.format(when=when)
# m.save(path)


In [38]:
import pandas as pd
import folium
from datetime import datetime

now = datetime.now()

when = now.strftime('%Y-%m-%d %H:%M:%S')
when

# Load your DataFrame (ctc_df)
# ctc_df = pd.read_csv('your_data.csv')
ctc_df = sorted_score_df.head(188)

# Assuming you have the 'outer' DataFrame with latitude and longitude columns
# outer = pd.read_csv('outer_data.csv')

# Convert latitude and longitude columns to float if needed
ctc_df['위도'] = ctc_df['위도'].astype(float)
ctc_df['경도'] = ctc_df['경도'].astype(float)
outer['위도'] = outer['위도'].astype(float)
outer['경도'] = outer['경도'].astype(float)

# Calculate the mean latitude and longitude for centering the map
center_lat = (ctc_df['위도'].mean() + outer['위도'].mean()) / 2
center_lon = (ctc_df['경도'].mean() + outer['경도'].mean()) / 2

# Create a Folium map centered at the mean location
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Add markers for ctc_df with blue color
for index, row in outer.iterrows():
    folium.Marker(
        location=[row['위도'], row['경도']],
        popup=row['정류장명'],  # Optional popup information
        icon=folium.Icon(color='red')  # Blue icon
    ).add_to(m)

# Add markers for outer with red color
for index, row in ctc_df.iterrows():
    folium.Marker(
        location=[row['위도'], row['경도']],
        popup=row['정류장명'],  # Optional popup information
        icon=folium.Icon(color='blue')  # Red icon
    ).add_to(m)

# Save the map to an HTML file
path = '/Users/iseunghyeon/Desktop/프로젝트/data/folium시각화/{when}.html'.format(when=when)
m.save(path)
